In [1]:
import sys
sys.path.append("../utils/") 
from utilities import performance_metrics
import time

from src.core import Term, Atom
from src.ilp import Language_Frame, Program_Template, Rule_Template
from src.dilp import DILP
import tensorflow as tf
import numpy as np
from src.utils import process_file, process_dir, create_table, output_rules, test_rule, train, test
from src.generate_template import create_templates
import duckdb
tf.random.set_seed(1000)
np.random.seed(1000)
import dill


2024-05-30 18:28:32.996281: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-30 18:28:34.672907: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
con = duckdb.connect(':memory:')
# enable automatic query parallelization
con.execute("PRAGMA threads=2")
# enable caching of parquet metadata
con.execute("PRAGMA enable_object_cache")

Main idea: We need to learn a rule: 

Fraud_trans(U1, U2):- Transaction(X), Orig(X, U1), Dest(X, U2),  Fraud(X).

Fraud_chain(U1, U2):- Fraud_trans(U1, Y), Fraud_chain(Y, U2),

Fraud_trans(U1, U2):-  pred2(u1, u2), Fraud(X).
pred2(u1,u2) = Orig(X, U1), Dest(X, U2), 

In [4]:
term_x_0 = Term(True, 'X_0')
term_x_1 = Term(True, 'X_1')
input_dir = "../examples/chain-fraud/"
input_table = create_table(con, input_dir)
print(f'ratio of positives %f' % (100*input_table['Target'].sum()/len(input_table)))
target, p_e, constants, B, P, N = process_dir(input_dir)
p_e

ratio of positives 20.000000
113
end of facts processing
6
end of positive examples processing
24
end negative examples processing
data is in order


[A(X_0),
 To_destination(X_0,X_1),
 Cons_orig(X_0,X_1),
 C(X_0),
 B(X_0),
 From_orig(X_0,X_1)]

In [8]:


p_a, rules = create_templates(p_e, target, term_x_0)
rules[target] = (rules[target][0], Rule_Template(v=1, allow_intensional=True))
Fraud_trans = Atom([term_x_0, term_x_1], 'Fraud_trans')
Fraud_chain = Atom([term_x_0, term_x_1], 'Fraud_chain')
p_a.append(Fraud_trans)
p_a.append(Fraud_chain)
rules[Fraud_trans] = (Rule_Template(v=1, allow_intensional=True), None) 
rules[Fraud_chain] = (Rule_Template(v=1, allow_intensional=True), None) 

language_frame = Language_Frame(target, p_e, constants)
# program_template = Program_Template(p_a, rules, T=10)
program_template = Program_Template(p_a, rules, T=6)

previous table dropped
ratio of positives 15.662651
314
end of facts processing
13
end of positive examples processing
70
end negative examples processing
data is in order


In [9]:
rules

{pred1(X_0): (<src.ilp.template.rule_template.Rule_Template at 0x7f877a309b90>,
  None),
 pred2(X_0): (<src.ilp.template.rule_template.Rule_Template at 0x7f877a3407d0>,
  None),
 pred3(X_0): (<src.ilp.template.rule_template.Rule_Template at 0x7f877a341f90>,
  None),
 Fraud(X_0): (<src.ilp.template.rule_template.Rule_Template at 0x7f877a342590>,
  <src.ilp.template.rule_template.Rule_Template at 0x7f877a342610>),
 Fraud_trans(X_0,X_1): (<src.ilp.template.rule_template.Rule_Template at 0x7f877a3417d0>,
  None),
 Fraud_chain(X_0,X_1): (<src.ilp.template.rule_template.Rule_Template at 0x7f877a343910>,
  None)}

In [6]:
print("DILP initialisation")
dilp = DILP(language_frame, B, P, N, program_template, allow_target_recursion=True)
start_time = time.time()
dilp.train()
finish_time = time.time()
print("execution time %d" % (finish_time - start_time))

DILP initialisation
Performing Inference
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Inference Complete
Performing Inference
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Inference Complete
--------------------
step 0 loss is 9.065052032470703
Performing Inference
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Inference Complete
Fraud(17) Expected: 0 1.000
Fraud(20) Expected: 0 0.988
Fraud(80) Expected: 0 1.000
Fraud(11) Expected: 0 1.000
Fraud(18) Expected: 0 0.988
Fraud(61) Expected: 0 1.000
Fraud(81) Expected: 0 1.000
Fraud(6) Expected: 0 1.000
Fraud(8) Expected: 0 1.000
Fraud(93) Expected: 0 1.000
Fraud(78) Expected: 0 1.000
Fraud(24) Expected: 0 1.000
Fraud(94) Expected: 0 0.997
Fraud(55) Expected: 0 1.000
Fraud(31) Expected: 0 1.000
Fraud(37) Expected: 0 0.997
Fraud(62) Expected: 0 1.000
Fraud(47) Expected: 0 1.000
Fraud(2) Expected: 0 1.000
Fraud(75) Expected: 1 0.997
Fraud(98) Ex

KeyboardInterrupt: 

In [39]:
abcd_rules = dilp.show_definition()
sql_str = output_rules(abcd_rules)
input_table = create_table(con, input_dir)
# predicted_table = test_rule(con, sql_str, target_predicate="Target")
# performance_metrics(predicted_table["Target"],input_table["Target"], labels=[True,False])

----------------------------
Target(X_0)
Target(X_0) -> pred1(X_0),B(X_0)
Target(X_0) -> Cons_orig(X_0,X_1),Target(X_1)
----------------------------
----------------------------
pred1(X_0)
pred1(X_0) -> B(X_0),pred4(X_0)
None
----------------------------
----------------------------
pred2(X_0)
pred2(X_0) -> B(X_0),pred4(X_0)
None
----------------------------
----------------------------
pred3(X_0)
pred3(X_0) -> pred2(X_0),C(X_0)
None
----------------------------
----------------------------
pred4(X_0)
pred4(X_0) -> A(X_0),C(X_0)
None
----------------------------
pred4 : A C
pred3 : pred2 C
pred2 : B pred4
pred1 : B pred4
Target : pred1 B
Target : Cons_orig Target
select A and C as pred4,
 pred2 and C as pred3,
 B and pred4 as pred2,
 B and pred4 as pred1,
 pred1 and B as Target,
 Cons_orig and Target as Target,

previous table dropped


In [41]:
print(*zip(rules.keys(), map(lambda x: (x[0].v, x[0].allow_intensional), list(rules.values()))))
rules

(pred1(X_0), (0, True)) (pred2(X_0), (0, True)) (pred3(X_0), (0, False)) (Target(X_0), (0, True))


{pred1(X_0): (<src.ilp.template.rule_template.Rule_Template at 0x7ff4e99972d0>,
  None),
 pred2(X_0): (<src.ilp.template.rule_template.Rule_Template at 0x7ff513a95ed0>,
  None),
 pred3(X_0): (<src.ilp.template.rule_template.Rule_Template at 0x7ff513886f90>,
  None),
 Target(X_0): (<src.ilp.template.rule_template.Rule_Template at 0x7ff513887410>,
  None)}

In [43]:
T=2
p_a, rules = create_templates(p_e[:-1], target, term_x_0)
rules[target] = (rules[target][0], Rule_Template(v=1, allow_intensional=True))

program_template = Program_Template(p_a, rules, T=T)
dilp = DILP(language_frame, B, P, N, program_template, allow_target_recursion=True)
train(dilp)


Performing Inference
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Inference Complete
Performing Inference
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Inference Complete
--------------------
step 0 loss is 0.685841977596283
Performing Inference
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Inference Complete
Target(4) Expected: 1 0.072
Target(3) Expected: 0 0.007
Target(18) Expected: 1 0.011
Target(12) Expected: 0 0.007
Target(9) Expected: 0 0.011
Target(0) Expected: 0 0.037
Target(25) Expected: 0 0.021
Target(19) Expected: 0 0.033
Target(11) Expected: 0 0.037
Target(10) Expected: 0 0.021
Target(26) Expected: 1 0.072
Target(8) Expected: 0 0.033
Target(23) Expected: 0 0.037
Target(24) Expected: 0 0.037
Target(16) Expected: 0 0.037
Target(6) Expected: 0 0.011
Target(21) Expected: 1 0.072
Target(2) Expected: 1 0.072
Target(29) Expected: 0 0.015
Target(17) Expected: 0 0.015
Target(14) Expec

In [9]:
dill.dump_session('notebook_chain_recursion.db')

In [45]:
T=3
p_a, rules = create_templates(p_e[:-1], target, term_x_0)
rules[target] = (rules[target][0], Rule_Template(v=1, allow_intensional=True))

program_template = Program_Template(p_a, rules, T=T)
dilp = DILP(language_frame, B, P, N, program_template, allow_target_recursion=True)
train(dilp)

Performing Inference
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Inference Complete
Performing Inference
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Inference Complete
--------------------
step 0 loss is 0.5645529627799988
Performing Inference
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Inference Complete
Target(4) Expected: 1 0.140
Target(3) Expected: 0 0.018
Target(18) Expected: 1 0.028
Target(12) Expected: 0 0.018
Target(9) Expected: 0 0.032
Target(0) Expected: 0 0.070
Target(25) Expected: 0 0.066
Target(19) Expected: 0 0.083
Target(11) Expected: 0 0.070
Target(10) Expected: 0 0.066
Target(26) Expected: 1 0.140
Target(8) Expected: 0 0.083
Target(23) Expected: 0 0.070
Target(24) Expected: 0 0.070
Target(16) Expected: 0 0.070
Target(6) Expected: 0 0.032
Target(21) Expected: 1 0.140
Target(2) Expected: 1 0.140
Target(29) Expected: 0 0.043
Target(17) Expected: 0 0.043
Target(14) Expe

In [46]:
T=5
program_template = Program_Template(p_a, rules, T=T)
dilp = DILP(language_frame, B, P, N, program_template, allow_target_recursion=True)
train(dilp)

Performing Inference
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Inference Complete
Performing Inference
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Inference Complete
--------------------
step 0 loss is 0.4849053919315338
Performing Inference
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Inference Complete
Target(4) Expected: 1 0.494
Target(3) Expected: 0 0.063
Target(18) Expected: 1 0.093
Target(12) Expected: 0 0.063
Target(9) Expected: 0 0.235
Target(0) Expected: 0 0.161
Target(25) Expected: 0 0.348
Target(19) Expected: 0 0.352
Target(11) Expected: 0 0.162
Target(10) Expected: 0 0.348
Target(26) Expected: 1 0.494
Target(8) Expected: 0 0.352
Target(23) Expected: 0 0.162
Target(24) Expected: 0 0.161
Target(16) Expected: 0 0.162
Target(6) Expected: 0 0.235
Target(21) Expected: 1 0.494
Target(2) Expected: 1 0.494
Target(29) Expected: 0 0.093
Target(17) Expected: 0 0.093
Target(14) Expe

In [47]:
import dill
dill.dump_session('notebook_chain_recursion_multiple_T.db')

TypeError: cannot pickle 'duckdb.duckdb.DuckDBPyConnection' object